# 06-measure-novelty-from-aggregated-topics

In [9]:
import os
import glob
import numpy as np
import pandas as pd
from itertools import *

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.rcParams['figure.figsize'] = [7,8]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 200

mpl.rcParams['font.size'] = 17
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['lines.markersize'] = 10

sns.set_context('poster')

from pprint import pprint
from dateutil.parser import parse
from datetime import datetime
import time

import imp

from scipy.stats import entropy
entropy_fix = np.log2(np.e)

In [10]:
def calc_week(x):
    d1 = parse(x)
    d0 = datetime(d1.year,1,1,0,0)
    d_gap = d1-d0
    week = d_gap.days/7 + 1
    return int(week)

In [11]:
df_allnews = pd.DataFrame()
OCR_folder = 'scraping_times/data/OCRtext/'
newscodes_folder = 'scraping_times/data/codes_for_only_News/'
#columns = ['pubdate', 'headline', 'text', 'filename']
columns = ['pubdate', 'filename']

min_file_size = 1000 # bytes
yearrange = range(1947,2013)

for year in yearrange:
    
    newscodes_filenames = glob.glob(newscodes_folder + str(year) + '*')
    filecodes_news = []
    for infile in newscodes_filenames:
        with open(infile) as f:
            filecodes_news += f.read().split()
    
    all_infiles = glob.glob(OCR_folder + str(year) + '*/*')
    to_df = []
    for infile in all_infiles:
        file_size = os.stat(infile).st_size # in bytes
        if file_size > min_file_size:
            
            with open(infile) as f:
                lines = [ i[:-1] for i in f.readlines()[2:] ]
                headline = lines[0]
                pubdate = infile.split('/')[3]
                text = ' '.join(lines[1:])
                filename = pubdate+'/'+infile.split('/')[-1]
              
                filecode = filename.split('_')[-1][:-4]
                IS_NEWS = ( filecode in filecodes_news )
                HAS_ENOUGH_TEXT = (len(text) > 50)
                
                if IS_NEWS and HAS_ENOUGH_TEXT:
                    #to_df += [( pubdate, headline, text, filename )]
                    to_df += [( pubdate, filename )]
                    
    print("%d newspieces in %d" % (len(to_df),year))                
    df_allnews = df_allnews.append(pd.DataFrame(to_df,columns=columns),ignore_index=True)

df_allnews['year']  = df_allnews['pubdate'].apply( lambda x: int(x[:4]) )
df_allnews['month'] = df_allnews['pubdate'].apply( lambda x: int(x[5:7]) )
df_allnews['week']  = df_allnews['pubdate'].apply( lambda x: calc_week(x) ) 
    
print("%d newspieces from %d to %d" % (len(df_allnews), min(yearrange), max(yearrange)) )
df_allnews.head()

988 newspieces in 1947
732 newspieces in 1948
1035 newspieces in 1949
2018 newspieces in 1950
1915 newspieces in 1951
2466 newspieces in 1952
2532 newspieces in 1953
2555 newspieces in 1954
2369 newspieces in 1955
1160 newspieces in 1956
1650 newspieces in 1957
1250 newspieces in 1958
1706 newspieces in 1959
1385 newspieces in 1960
2543 newspieces in 1961
1828 newspieces in 1962
2575 newspieces in 1963
1885 newspieces in 1964
3231 newspieces in 1965
2031 newspieces in 1966
3125 newspieces in 1967
1861 newspieces in 1968
3054 newspieces in 1969
1906 newspieces in 1970
1785 newspieces in 1971
2057 newspieces in 1972
1829 newspieces in 1973
1773 newspieces in 1974
1885 newspieces in 1975
2007 newspieces in 1976
3727 newspieces in 1977
2214 newspieces in 1978
81 newspieces in 1979
1738 newspieces in 1980
2709 newspieces in 1981
2441 newspieces in 1982
3375 newspieces in 1983
2798 newspieces in 1984
2794 newspieces in 1985
3056 newspieces in 1986
2376 newspieces in 1987
3124 newspieces in 1

,pubdate,filename,year,month,week
0,1947-04-30,1947-04-30/102_GALE_CS119096478.txt,1947,4,18
1,1947-04-30,1947-04-30/076_GALE_CS85279902.txt,1947,4,18
2,1947-04-30,1947-04-30/103_GALE_CS119227550.txt,1947,4,18
3,1947-04-30,1947-04-30/055_GALE_CS68502686.txt,1947,4,18
4,1947-04-30,1947-04-30/117_GALE_CS134431902.txt,1947,4,18


In [12]:
n_topics = 50  # 10, 15, 20, 25, 50
topics_per_document = pd.read_csv('times_data/document_topic_distributions_'+str(n_topics)+'topics.csv', index_col=0)

## Aggregate topic distributions per period

In [13]:
from datetime import datetime
import time

to_date = lambda date: datetime.fromtimestamp(time.mktime(time.strptime('{} {} 1'.format(date[0],date[1]), '%Y %W %w'))) 

doc_to_date = { row.filename: (to_date((row.year, row.week)), row.week)
                for _, row in df_allnews.iterrows() }

In [14]:
all_dates = set(sorted(doc_to_date.values()))
all_documents = topics_per_document.index.values

In [15]:
period_names = ['week','month','quarter','year']
agg_topics = { pn:{} for pn in period_names }

i=0
for doc_id, dates in doc_to_date.items():
    i+=1
    if i % 20000 == 0:
        print('{}/{} done'.format(i,len(doc_to_date)))
        
    year   = dates[0].year 
    quarter= (dates[0].month-1)//3
    month  = dates[0].month
    week   = dates[1]
    
    year_s    = str(year)
    quarter_s = str(year)+'-'+str(quarter).zfill(2)
    month_s   = str(year)+'-'+str(month).zfill(2)
    week_s    = str(year)+'-'+str(week).zfill(2)
    
    try:
        if len(topics_per_document.loc[doc_id]) < n_topics:
            topics = topics_per_document.loc[doc_id].iloc[0].values
        else:
            topics = topics_per_document.loc[doc_id].values

        periods = [week_s,month_s,quarter_s,year_s]
        for period_name, period in zip(period_names, periods):
            if period not in agg_topics[period_name]:
                agg_topics[period_name][period]  = topics
            else:
                agg_topics[period_name][period] += topics
                
    except KeyError as e:
        print(e, 'not found')

'1951-10-24/044_GALE_CS52514136.txt' not found
'1954-06-16/028_GALE_CS69030608.txt' not found
20000/157205 done
'1963-03-13/063_GALE_CS102327405.txt' not found
'1966-11-23/002_GALE_CS17263479.txt' not found
40000/157205 done
'1975-11-26/138_GALE_CS269319546.txt' not found
60000/157205 done
'1977-03-02/145_GALE_CS268664930.txt' not found
'1978-05-10/142_GALE_CS268665514.txt' not found
'1982-04-21/007_GALE_CS17926805.txt' not found
'1982-04-21/092_GALE_CS151227029.txt' not found
'1983-06-15/012_GALE_CS18582735.txt' not found
80000/157205 done
100000/157205 done
120000/157205 done
'2000-06-07/068_GALE_IF0501357530.txt' not found
'2000-02-02/376_GALE_IF0501002958.txt' not found
'2002-02-27/393_GALE_IF0501480732.txt' not found
'2002-02-20/064_GALE_IF0501101607.txt' not found
140000/157205 done


In [16]:
newspaper = 'Times'
period_names = ['week','month','quarter','year']

for pn in period_names:
    topics = agg_topics[pn]   
    outfile = '../Null-models/data/aggregate_{}topics_per_{}_{}.csv'.format(n_topics,pn,
                                                                            newspaper)
    df = pd.DataFrame(topics.values(), index=topics.keys())
    df.to_csv(outfile)

## Read them again

In [17]:
newspaper = 'Times'
period_names = ['week','month','quarter','year']
agg_dfs = {}

for pn in period_names:
    infile = '../Null-models/data/aggregate_{}topics_per_{}_{}.csv'.format(n_topics,pn,
                                                                           newspaper)
    df = pd.read_csv(infile, index_col=0)
    agg_dfs[pn] = df.sort_index()

In [18]:
# Novelty measures

from numpy.linalg import norm
from scipy.stats import entropy
from scipy.spatial.distance import hamming
from scipy.spatial.distance import euclidean
from sklearn.metrics import mutual_info_score

def rel_entr(P, Q):                
    return entropy(P,Q)*entropy_fix

def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (rel_entr(_P, _M) + rel_entr(_Q, _M))

def BCD(P,Q):
    _P = np.array(P / norm(P, ord=1),dtype=np.float32)
    _Q = np.array(Q / norm(Q, ord=1),dtype=np.float32)
    BC = np.dot(np.sqrt(_P),np.sqrt(_Q))
    return -np.log2(BC)    

def MI(_P,_Q):
    return mutual_info_score(_P,_Q)

def novelty(p,q, metric='KL'):
    
    if metric=='KL':
        return rel_entr(p,q)
    elif metric=='hamming':
        return hamming(p>0,q>0)
    elif metric=='euclidean':
        return euclidean(p,q)
    elif metric=='JSD':
        return JSD(p,q)
    elif metric=='BCD':
        return BCD(p,q)
    elif metric=='MI':
        return MI(p,q)
    else:
        return 0

In [19]:
KLs = { pn:[] for pn in period_names }

for pn in period_names:
    df = agg_dfs[pn]
    for t,tm1 in zip(df.index[1:],df.index[:-1]):
        topics_t   = df.loc[t]/df.loc[t].sum()
        topics_tm1 = df.loc[tm1]/df.loc[tm1].sum()
        KL = novelty(topics_t, topics_tm1)
        KLs[pn] += [(t,KL)]

In [20]:
newspaper = 'Times'
period_names = ['week','month','quarter','year']

for pn in period_names:
    outfile = '../Null-models/data/agg_novelty_{}topics_per_{}_{}.csv'.format(n_topics,pn,
                                                                              newspaper)
    print('Printing', outfile)
    with open(outfile, 'w') as f:
        f.write(''.join(['{},{}\n'.format(p[0],p[1]) for p in KLs[pn]]))

Printing ../Null-models/data/agg_novelty_50topics_per_week_Times.csv
Printing ../Null-models/data/agg_novelty_50topics_per_month_Times.csv
Printing ../Null-models/data/agg_novelty_50topics_per_quarter_Times.csv
Printing ../Null-models/data/agg_novelty_50topics_per_year_Times.csv
